In [1]:
%cd /content
# %pip install -r /content/requirments.txt

/content


/pyenv/versions/3.10.14/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# Setup environment
import os
from dotenv import load_dotenv
from pprint import pprint as pp

load_dotenv()
print(os.environ['ZAPIER_WEBHOOK'])

import jax
jax.devices()

https://hooks.zapier.com/hooks/catch/6996241/23u9vlk/


[rocm(id=0)]

In [3]:
import keras_nlp
# keras.config.set_floatx("float16")
# Model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
# gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
#     proto="/tokenizer/gemma_ua_ordered.model"
# )
# gemma_lm.load_weights("../chkpts/weights.h5")

2024-07-04 09:51:11.069147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-04 09:51:12.964717: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:1313] failed to allocate 181.43GiB (194814515456 bytes) from device: HIP_ERROR_OutOfMemory
2024-07-04 09:51:12.964761: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:1313] failed to allocate 163.29GiB (175333048320 bytes) from device: HIP_ERROR_OutOfMemory
2024-07-04 09:51:12.964772: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:1313] failed to allocate 146.96GiB (157799743488 bytes) from device: HIP_ERROR_OutOfMemory
2024-07-04 09:51:12.964781: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:1313] failed to allocate 132.27GiB (142019772416 bytes) from device: HIP_ERROR_OutOfMemory
2024-07-04 09:51:12.964791: E external/xla/xla/stream_executor/rocm/rocm_driver.cc:1313] failed to allocate 119.04GiB (127817793536 bytes) from device: HIP_ERROR_OutOfMemory
normal

In [4]:
test_prompts_dev = [
    """Борщ це -""",
    """Крим - це територія"""
]

In [5]:
with open('input_prompts.txt', 'r') as file:
    test_prompts = [line.rstrip('\n') for line in file]

def process_prompts():
    llm_outputs = []
    result_str = ""
    for prompt in test_prompts:
        result_str += f"PROMPT:\"{prompt}\"\n"
        output = gemma_lm.generate(prompt, max_length=96)
        llm_outputs.append(output)
        result_str += f"LLM:\n{output}\n\n"
    return result_str

In [6]:
# Zapier snippet
import requests
from datetime import datetime
webhook_url = os.environ["ZAPIER_WEBHOOK"]
model_metadata = "blank metadata"
body = "blank body"
def post_to_slack():
    requests.post(webhook_url, json={'time': str(datetime.now().isoformat()), 'metadata': model_metadata, 'body': body, 'server': 'AMD mi300x'})

In [7]:
model_metadata = f"gemma-7b base model vanilla"
body = process_prompts()
post_to_slack()

2024-07-04 09:51:39.330122: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


In [9]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_7b_en")
gemma_lm.preprocessor.tokenizer = keras_nlp.models.GemmaTokenizer(
    proto="/tokenizer/gemma_ua_ordered.model"
)
model_metadata = f"gemma-7b base model with custom UA tokenizer"
body = process_prompts()
post_to_slack()

In [12]:
gemma_lm.load_weights("/ckpts/ckpt_meaninit_full_step_260000_loss_0.99.weights.h5")
model_metadata = f"gemma-7b base model with custom UA tokenizer and embd trained on 1B wiki-ua dataset"
body = process_prompts()
post_to_slack()

In [26]:
folder_path = "/tmp/output"
file_names = os.listdir(folder_path)
print(len(file_names), sorted(file_names))
percents = [35, 50, 70, 85, 100, 130]
for elem in zip(percents, sorted(file_names)):
    print(f"{elem[0]}% of {elem[1]}")
    gemma_lm.load_weights(f"/tmp/output/{elem[1]}")
    model_metadata = f"{elem[0]}% of 1 epoch pretraining on 6B ua-wiki starting from trained embd"
    body = process_prompts()
    post_to_slack()


6 ['ckpt_step_13200_loss_10.81.weights.h5', 'ckpt_step_19200_loss_13.06.weights.h5', 'ckpt_step_25200_loss_14.88.weights.h5', 'ckpt_step_31200_loss_15.94.weights.h5', 'ckpt_step_37200_loss_15.94.weights.h5', 'ckpt_step_48600_loss_15.94.weights.h5']
35% of ckpt_step_13200_loss_10.81.weights.h5
50% of ckpt_step_19200_loss_13.06.weights.h5
70% of ckpt_step_25200_loss_14.88.weights.h5
85% of ckpt_step_31200_loss_15.94.weights.h5
100% of ckpt_step_37200_loss_15.94.weights.h5
130% of ckpt_step_48600_loss_15.94.weights.h5
